# Notebook: Create Subset

This notebook is used to create a subset of **2000** tweets, which will then be annotated with respect to their sentiment.
<br>**Contributors:** [Nils Hellwig](https://github.com/NilsHellwig/) | [Markus Bink](https://github.com/MarkusBink/)

## Packages

In [1]:
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import random
import os

## Parameters

In [2]:
ANNOTATION_DATASET_PATH = '../Datasets/annotation_dataset'
DATASET_PATH = '../Datasets/dataset/'
SUBSET_SIZE = 2000
SEED_VALUE = 0
PARTIES = ["CDU_CSU", "SPD", "AFD", "FDP", "GRUENE", "LINKE"]

## Code

### 1. Get Reproducable Results

In [3]:
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)

### 2. Calculate Number of Tweets crawled for each party

In [4]:
n_tweets_total = 0
party_statistics = {}

In [5]:
for party in PARTIES:
    n_tweets_party = 0
    for subdir, _, files in os.walk(DATASET_PATH + party):
        for file in files:
            if file.endswith('.csv') and subdir[len(DATASET_PATH):] in PARTIES:
                # Get username of CSV file
                username = file[:-4]

                # Read dataframe
                df = pd.read_csv(DATASET_PATH + party + "/" + file, sep=",", index_col=0)

                # Add to counter
                n_tweets_party += df.shape[0]

                # Add length to n_tweets_total
                n_tweets_total += df.shape[0]

    party_statistics[party] = n_tweets_party

In [6]:
n_tweets_total

707241

In [7]:
party_statistics

{'CDU_CSU': 227683,
 'SPD': 228415,
 'AFD': 57572,
 'FDP': 79815,
 'GRUENE': 73261,
 'LINKE': 40495}

### 3. Check which party gets an additional Tweet / if tweet needs to be removed

In [8]:
def get_key_with_max_value_under_0_5(dictionary):
    filtered_dict = {}
    for key, value in dictionary.items():
        if value < 0.5:
            filtered_dict[key] = value
    return max(filtered_dict, key=filtered_dict.get)

In [9]:
def truncate(x, d):
    return int(x*(10.0**d))/(10.0**d)

for party in party_statistics:
    party_statistics[party] = ((SUBSET_SIZE / n_tweets_total) * party_statistics[party])

In [10]:
party_statistics

{'CDU_CSU': 643.8625588731422,
 'SPD': 645.932574610352,
 'AFD': 162.8073033096215,
 'FDP': 225.70806839535604,
 'GRUENE': 207.1740750324147,
 'LINKE': 114.51541977911349}

### 4. Get Random Tweets From Each Account

In [11]:
n_subset_total = 0

In [ ]:
annotation_dataset = pd.DataFrame()
for party in PARTIES:
    # Initialize an empty DataFrame to store the tweets from accounts of a party
    df_party = pd.DataFrame()

    for subdir, _, files in os.walk(DATASET_PATH + party):
        for file in files:
            if file.endswith('.csv') and subdir[len(DATASET_PATH):] in PARTIES:
                # Get username of CSV file
                username = file[:-4]

                # Read dataframe
                df_account = pd.read_csv(DATASET_PATH + party + "/" + file, sep=",", index_col=0)

                # Add dataframe to party dataframe
                df_party = pd.concat([df_party, df_account], axis=0).reset_index().drop(columns='index')

    n_tweets_party = df_party.shape[0]
    n_tweets_party_for_subset = round((SUBSET_SIZE / n_tweets_total) * n_tweets_party)

    n_subset_total += n_tweets_party_for_subset

    df_samples_for_party = df_party.sample(n=n_tweets_party_for_subset, random_state=SEED_VALUE)
    annotation_dataset = pd.concat([annotation_dataset, df_samples_for_party], axis=0).reset_index().drop(columns='index')      

In [ ]:
n_subset_total

### 4. Create Sub Datasets for Annotation 

Save dataset for annotation

In [ ]:
try:
    os.makedirs(ANNOTATION_DATASET_PATH)
except FileExistsError:
    pass

In [ ]:
annotation_dataset = annotation_dataset.sample(frac=1, random_state=SEED_VALUE).reset_index().drop(columns='index')

By rounding the number of tweets, it is possible that not exactly 2000 tweets will come out. In this case, one tweet must be removed.

In [ ]:
party_to_exclude_tweets = annotation_dataset[annotation_dataset['source_party'] == 'LINKE']
tweet_to_delete = party_to_exclude_tweets.sample(n=1)
annotation_dataset = annotation_dataset.drop(tweet_to_delete.index).reset_index().drop(columns='index')

In [ ]:
annotation_dataset['source_party'].value_counts()

In [ ]:
tweet_to_delete

In [ ]:
annotation_dataset

Save entire annotation_dataset (unlabled) as CSV

In [ ]:
annotation_dataset.to_csv(ANNOTATION_DATASET_PATH + "/annotation_dataset.csv")

In [ ]:
annotation_dataset['source_party'].value_counts()

For the whole corpus, we do not delete duplicates. Duplicates can occur because a tweet can mention several politicians at once, which means that a tweet could be crawled for multiple politicians. However, we want to avoid evaluating the trained BERT model with tweets that were also used for training. Therefore, we make sure that there are no duplicates among the 2000 annotated tweets that we will later use for training and evaluation of our BERT model.

In [ ]:
# Check if the 'id' column is unique
is_unique = df['id'].is_unique
print("Dataset uniqueness: ", is_unique)

Add column for sentiment label and columns with information that might be helpfull for annotators

In [ ]:
annotation_dataset["sentiment"] = ""
annotation_dataset = annotation_dataset.loc[:, ['id', 'username', 'date', 'sentiment', 'tweet', 'link', 'source_account', 'source_party']]

In [ ]:
df_session1 = annotation_dataset[:int(SUBSET_SIZE/2)]
df_session2 = annotation_dataset[int(SUBSET_SIZE/2):]

In [ ]:
df_session1.to_csv(ANNOTATION_DATASET_PATH + "/tweets_session_1.csv")
df_session1.to_excel(ANNOTATION_DATASET_PATH + "/tweets_session_1.xlsx")
df_session1

In [ ]:
df_session2.to_csv(ANNOTATION_DATASET_PATH + "/tweets_session_2.csv")
df_session2.to_excel(ANNOTATION_DATASET_PATH + "/tweets_session_2.xlsx")
df_session2

## IMPORTANT: NEXT STEPS

1. Create new Folder "annotated_datasets" in `/Datasets`
2. Add Annotated Datasets in .xlsx format 
3. Name these:
`
['../Datasets/annotated_dataset/tweets_session_1_1.xlsx',
 '../Datasets/annotated_dataset/tweets_session_1_2.xlsx',
 '../Datasets/annotated_dataset/tweets_session_1_3.xlsx',
 '../Datasets/annotated_dataset/tweets_session_2_1.xlsx',
 '../Datasets/annotated_dataset/tweets_session_2_2.xlsx',
 '../Datasets/annotated_dataset/tweets_session_2_3.xlsx']
`

Schema:

`
../Datasets/annotated_dataset/tweets_session_{SESSION_ID}_{ANNOTATOR_ID}.xlsx'
`